In [19]:
import pandas as pd
from sqlalchemy import create_engine
import datetime as dt
import matplotlib.pyplot as plt

### Store CSV into DataFrame

In [3]:
csv_file = "./Resources/CBBTCUSD.csv"
Bitcoin_df = pd.read_csv(csv_file)
Bitcoin_df.head()

,DATE,CBBTCUSD
0,8/19/15,227.34
1,8/20/15,235.56
2,8/21/15,232.85
3,8/22/15,231.15
4,8/23/15,229.21


In [4]:
csv_file_2 = "./Resources/Ethereum Historical Data.csv"
Ethereum_df = pd.read_csv(csv_file_2)
Ethereum_df.head()

,Date,Price,Open,High,Low,Vol.,Change %
0,"Aug 02, 2020",368.23,387.15,415.08,335.76,16.78M,-4.88%
1,"Aug 01, 2020",387.14,346.48,392.37,343.33,11.03M,11.73%
2,"Jul 31, 2020",346.51,334.84,349.34,329.02,8.69M,3.48%
3,"Jul 30, 2020",334.87,317.79,341.11,314.83,9.07M,5.36%
4,"Jul 29, 2020",317.82,316.95,324.93,312.64,8.72M,0.27%


### Create new data with select columns

In [5]:
Bitcoin_exct = Bitcoin_df[['DATE', 'CBBTCUSD']].copy()
Bitcoin_exct.head()

,DATE,CBBTCUSD
0,8/19/15,227.34
1,8/20/15,235.56
2,8/21/15,232.85
3,8/22/15,231.15
4,8/23/15,229.21


In [6]:
Ethereum_exct = Ethereum_df[['Date','Price']].copy()
Ethereum_exct.head()

,Date,Price
0,"Aug 02, 2020",368.23
1,"Aug 01, 2020",387.14
2,"Jul 31, 2020",346.51
3,"Jul 30, 2020",334.87
4,"Jul 29, 2020",317.82


## Clean Data

In [7]:
# Altering date to date format
Ethereum_exct = Ethereum_df[['Date','Price']].copy()
Ethereum_exct.columns=['date', 'eth_price']
#create dictionary to assign numeric values to the months
month_dict={"Jan": 1, "Feb": 2,
            "Mar": 3, "Apr": 4,
            "May": 5, "Jun": 6,
            "Jul": 7, "Aug": 8,
            "Sep": 9, "Oct": 10,
            "Nov": 11, "Dec": 12
           }
#iterate over "Date" column to convert dates to yyyy-mm-dd
for x in range(len(Ethereum_exct)):
    try:
        #use .split to convert each date into an array containing the month, day, and year (in that order)
        day=Ethereum_exct.iloc[x,0].split(" ")
        #use the month_dict to convert months to thier numeric values
        m= month_dict[day[0]]
        #use .strip to remove the "," from the day value
        d= day[1].strip(",")
        d=int(d)
        #the year variable should already be in the right format
        y= int(day[2])
        #combine the m, d, and y variables to create a new date in the preferred format
        new_date= dt.date(year=y, month=m, day=d)
        #edit the dataframe with the new date
        Ethereum_exct.iloc[x,0]=new_date
    except:
        print("date already transformed or incorrectly formatted")
#Export to CSV file to load to SQL
#Ethereum_exct.to_csv("/Users/jonathanrocha/Desktop/ClassWork/ETL_Project/Resources/Ethereum_data.csv", index=False)
Ethereum_exct.head()

,date,eth_price
0,2020-08-02,368.23
1,2020-08-01,387.14
2,2020-07-31,346.51
3,2020-07-30,334.87
4,2020-07-29,317.82


## Connection from SQL

In [8]:
#Input for username to postgress
userinput = input("Enter your value: ") 

In [9]:
#Input for password to postgress
passInput = input("Enter your value: ") 

In [11]:
rds_connection_string = "@localhost:5432/crypto_db"
engine = create_engine(f'postgresql://{userinput}:{passInput}{rds_connection_string}')

In [12]:
#Read in database from SQL
bitcoin_df=pd.read_sql_query('select * from bitcoin', con=engine)

In [13]:
#Read in database from SQL
ethereum_df=pd.read_sql_query('select * from ethereum', con=engine)

In [58]:
#Verify dataframes
bitcoin_df.head()

,date,bit_price
0,2015-08-19,$227.34
1,2015-08-20,$235.56
2,2015-08-21,$232.85
3,2015-08-22,$231.15
4,2015-08-23,$229.21


In [59]:
#Verify dataframes 
ethereum_df.head()

,date,eth_price
0,2020-08-02,$368.23
1,2020-08-01,$387.14
2,2020-07-31,$346.51
3,2020-07-30,$334.87
4,2020-07-29,$317.82


In [99]:
#Merge dataframes
Crypto_df=pd.merge(bitcoin_df,ethereum_df,on="date", how="right")
Crypto_df.tail()

,date,bit_price,eth_price
1602,2020-07-29,"$11,100.39",$317.82
1603,2020-07-30,"$11,139.67",$334.87
1604,2020-07-31,"$11,372.76",$346.51
1605,2020-08-01,"$11,805.82",$387.14
1606,2020-08-02,"$11,050.03",$368.23


In [96]:
Crypto_df.dtypes

date         object
bit_price    object
eth_price    object
dtype: object

In [83]:
#Display DataFrame
Crypto_df

,date,bit_price,eth_price
0,2016-03-10,$415.98,$11.75
1,2016-03-11,$419.39,$11.95
2,2016-03-12,$410.58,$12.92
3,2016-03-13,$412.52,$15.07
4,2016-03-14,$415.02,$12.50
...,...,...,...
1602,2020-07-29,"$11,100.39",$317.82
1603,2020-07-30,"$11,139.67",$334.87
1604,2020-07-31,"$11,372.76",$346.51
1605,2020-08-01,"$11,805.82",$387.14
